In [263]:
from sklearn.base import BaseEstimator, ClassifierMixin

class NaiveBayesClassifier(BaseEstimator, ClassifierMixin):  
    def __init__(self):
        None

    def fit(self, X=None, y=None):
        label_p = np.array(X[X.columns[-5:]].apply(lambda x: np.sum(x) * 1., axis = 0).values)
        label_p = (label_p / np.sum(label_p))
        
        # "confusion" function: predicted in rows, true in cols
        confusion = np.zeros((5,5), float)
        
        # loss matrix
        #loss = np.full((5,5), 0., float)
        #np.fill_diagonal(loss, 1.)
        
        for _, row in y.iterrows():
            #print row['Text']
            yi = [] + [c for c in str(row['Text'])]
            ptotal = [1., 1., 1., 1., 1.]
            for yil in yi:
                score = None
                if yil in X.index:
                    score = X[X.index == yil][:1].reset_index().values
                    score = np.delete(score, 0).astype(float)
                    score = np.array(score)
                else:
                    # If no letter found, assume equally likelihood to have come out
                    # of any of the labels.
                    continue
                p = score + np.array([1., 1., 1., 1., 1.])
                # Obtain P(X = x | Y = y) by calculating per-category frequency
                # of current letter.
                p = p / label_p
                # Obtain P(X = x | Y = y) / P(X = x) by calculating P(X = x) which
                # is the marginal over Y.
                p = p / p.sum().astype(float)
                ptotal = np.multiply(ptotal, p)

            # hack: divide all elements by max and set all elements < 1 to 0
            # i.e. [0.1, 0.2, 0.3, 0.2, 0.2] -> [0.33, 0.66, 1, 0.66, 0.66] -> [0, 0, 1, 0, 0]
            #ptotal = np.multiply(ptotal, prior_adj)
            pred = np.divide(ptotal, np.amax(ptotal))
            pred = np.around(pred).argmax()
            confusion[pred, row['Category']] += 1
            # print((pred, row['Category'], row['Text']))
            category = np.zeros(5)
            category[row['Category']] = 1
            # print(total)
            
        print(confusion)
        loss_score = (np.trace(confusion) / np.sum(confusion))
        print(loss_score)
        return self
    
    def transform(self, X):
        return X

    def predict(self, X):
        return []
    
    def get_params(self, deep=True):
        return {}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            self.setattr(parameter, value)
        return self

In [266]:
from sklearn.pipeline import Pipeline, FeatureUnion
from NGramGenerator import *

Xf = pd.read_csv("data/train_set_x.csv")
T  = pd.read_csv("data/train_set_y.csv")
X  = pd.merge(Xf, T, on = 'Id')
Y  = pd.read_csv("data/test_set_x.csv")

ngram = NGramGenerator(1, True, 0, False)
nbayes = NaiveBayesClassifier()


p = Pipeline(
    [( 'ngram', ngram )] +
    [( 'nbayes', nbayes )]
)

#p.set_params(ngram__n=2)
A = p.fit_transform(X[:3000], X[:1000])


[[  57.   31.   30.    6.    6.]
 [   1.  401.   55.    8.    1.]
 [   0.   58.  156.    1.    1.]
 [   1.   23.    8.  114.    2.]
 [   0.    0.    2.    0.   38.]]
0.766
